In [4]:
#The following code is not running on github interpreter because it do not have the lr_utils library 
#neither the data.h5 dataset. But we maintain the idea

This is a simple tutorial of how to implement a logistic regrssion from scratch. The results are not the best and can be easily improved. The main idea is to understand how a logistic regression works and how its decision bounderies 
are computed. we use a 'cat non-cat' dataset to exemplify.

Given a dataset ("data.h5") containing images labeled as cat (y=1) or non-cat (y=0) with shapes (num_px, num_px, 3) where 3 is for the 3 channels (RGB). We will build a simple image-recognition algorithm that can correctly classify pictures as cat or non-cat.

For this project we are gonna use the following packages
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [h5py](http://www.h5py.org) is a common package to interact with a dataset that is stored on an H5 file.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.
- [PIL](http://www.pythonware.com/products/pil/) and [scipy](https://www.scipy.org/) are used here to test your model 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

%matplotlib inline

In [ ]:
#We start by loading the dataset, which is already divided into train and test sets
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [ ]:
# Example of a picture
index = 30
plt.imshow(train_set_x_orig[index])

In [ ]:
# We can verify the shape of our data by accessing its attribute .shape
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

For convinience its good to reshape our images into a numpy arry of shape (num_px $*$ num_px $*$ 3, 1), this way we stack the three channels on the top of each other and work with each image of our data as an array, not as a matrix.
(when we work with convolutional networks we will see how to use each channel as a different matrix)
For this we use the command reshape()
```python
X_flatten = X.reshape(X.shape[0], -1).T      # X.T is the transpose of X
```

In [ ]:
# Reshape the training and test examples

train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

# Our new input has the following dimensions

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

To represent color images, the red, green and blue channels (RGB) must be specified for each pixel, and so the pixel value is actually a vector of three numbers ranging from 0 to 255.

It is also useful to center and standardize your dataset, substracting the mean of the whole array from each example, and then divide each example by the standard deviation of the data. But for picture datasets, it is simpler and more convenient and works almost as well to just divide every row of the dataset by 255 (the maximum value of a pixel channel).

In [ ]:
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

Next we implement the sigmoid function, given by:
$sigmoid( w^T x + b) = \frac{1}{1 + e^{-(w^T x + b)}}$ 

In [ ]:
def sigmoid(z):
    return 1.0 /(1 + np.exp(-z)) # use of numpy to vectorize the function

We may initialize our parameters with zeros or with random numbers. For simplicity lets start with zeros.

In [ ]:
def initialize_with_zeros(dim):

    w = np.zeros((dim, 1))
    b = 0
    
    #verify if the dimensions are ok
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

The most amazing step of working if NN is the back propagation. Shortly it is given by applying the chain rule fro the output to the input of our NN.

We start propagating the inputs throught the network. As we are doing just one logistic regression it is lie a NN with a single layer;
$S = \sigma(w^T X + b) = (a^{(0)}, a^{(1)}, ..., a^{(m-1)}, a^{(m)})$

And the cost function:
$J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$

We get the gradient computing the first derivatives:

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(S-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [ ]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
    # w = weights
    # b = bias
    m = X.shape[1]
    
    S = sigmoid(np.dot(X.T,w) + b).T                            # compute activation, each image is a column at X
    #S = sigmoid(np.sum(X*w, axis=0) + b).T                     #multiply in line and than sum over X
    cost = - np.sum(Y*np.log(S)+(1-Y)*np.log(1-S)) / m           # compute cost
    
    dw = np.dot(X,(S-Y).T) / m
    db = np.sum(S-Y)/m

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    #dictionary with the values
    grads = {"dw": dw, db": db}
    
    return grads, cost

We now follow to minimize our cost function applying the update rule:

$ \theta = \theta - \alpha \text{ } d\theta$, where $\alpha$ is the learning rate.

In [ ]:
def optimize(w, b, X, Y, num_iterations, learning_rate):

    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]

        w = w - learning_rate*dw
        b = b - learning_rate*db
        if i % 100 == 0:
            costs.append(cost)
    
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    
    return params, grads, costs

Finally, we predict the new inputs using the parameters we have learned
$\hat{Y} = A = \sigma(w^T X + b)$

Convert the entries of a into 0 (if activation <= 0.5) or 1 (if activation > 0.5)

In [ ]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)

    A = sigmoid(np.dot(X.T, w) + b).T
    for i in range(A.shape[1]):
        Y_prediction[0,i] = 0 if A[0,i] <= 0.5 else 1
        pass
    
    assert(Y_prediction.shape == (1, m))    
    return Y_prediction

And, as final model...

In [ ]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    w, b = initialize_with_zeros(X_train.shape[0])

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)

    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

Let's also plot the cost function and the gradients.

In [ ]:
# Plot learning curve (with costs)
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()